In [49]:
import os
from utils.config import GRAPH_SAVE_FOLDER, DATA_FOLDER
from utils.plotter import plot_line_comparison


In [50]:
import pandas as pd

data_dir = os.path.join(DATA_FOLDER)
extra_tag = f"OneWeb_channel_specs_250MHz_10.5GHz"    
tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# Ensure the tagged folder exists
os.makedirs(tagged_folder, exist_ok=True)

df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [51]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"OneWeb_BGAN"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
# # df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [52]:
import pandas as pd
import re

# Step 1: Drop columns that are all zeros or all NaN
df = df.drop(columns=[col for col in df.columns if ((df[col] == 0) | (df[col].isna())).all()])

# Step 2: Get max satellite ID from column names
sat_ids = [int(match.group(1)) for col in df.columns if (match := re.match(r'LEO(\d+)_', col))]
max_sat_id = max(sat_ids) if sat_ids else -1

print("Max satellite ID found:", max_sat_id)

# Step 3: Track satellites with access
possible_sat_conn_ids = []

for i in range(max_sat_id + 1):
    sat_id = f'LEO{i}'

    # Check access columns
    sydney_access_col = f'{sat_id}_Sydney_Access'
    melbourne_access_col = f'{sat_id}_Melbourne_Access'

    # Check if access columns exist and have any True values
    # .fillna(False) ensures we treat NaN as False
    # .any() checks if there is at least one True value
    sydney_access_ok = sydney_access_col in df.columns and df[sydney_access_col].fillna(False).any()
    melbourne_access_ok = melbourne_access_col in df.columns and df[melbourne_access_col].fillna(False).any()

    if sydney_access_ok or melbourne_access_ok:
        possible_sat_conn_ids.append(sat_id)
    else:
        # Drop base satellite info if no access
        base_cols = [f'{sat_id}_Name', f'{sat_id}_Lat', f'{sat_id}_Lon', f'{sat_id}_Freq_Hz']
        existing_cols_to_drop = [col for col in base_cols if col in df.columns]
        df = df.drop(columns=existing_cols_to_drop)

print("Satellites with possible connection:", possible_sat_conn_ids)
print("length(Satellites with possible connection):", len(possible_sat_conn_ids))


Max satellite ID found: 720
Satellites with possible connection: ['LEO20', 'LEO21', 'LEO22', 'LEO23', 'LEO24', 'LEO25', 'LEO26', 'LEO27', 'LEO28', 'LEO60', 'LEO61', 'LEO62', 'LEO63', 'LEO64', 'LEO65', 'LEO66', 'LEO67', 'LEO68', 'LEO101', 'LEO102', 'LEO103', 'LEO104', 'LEO105', 'LEO106', 'LEO107', 'LEO553', 'LEO554', 'LEO555', 'LEO556', 'LEO557', 'LEO558', 'LEO592', 'LEO593', 'LEO594', 'LEO595', 'LEO596', 'LEO597', 'LEO598', 'LEO599', 'LEO632', 'LEO633', 'LEO634', 'LEO635', 'LEO636', 'LEO637', 'LEO638', 'LEO639', 'LEO640', 'LEO672', 'LEO673', 'LEO674', 'LEO675', 'LEO676', 'LEO677', 'LEO678', 'LEO679', 'LEO680', 'LEO712', 'LEO713', 'LEO714', 'LEO715', 'LEO716', 'LEO717', 'LEO718', 'LEO719', 'LEO720']
length(Satellites with possible connection): 66


In [53]:
# Collect all satellite-related columns NOT in possible_sat_conn_ids to drop
cols_to_drop = []

for col in df.columns:
    # Extract satellite id number from column name if it matches pattern
    match = re.match(r'(LEO\d+)_', col)
    if match:
        sat_id = match.group(1)
        # If satellite not in possible_sat_conn_ids, mark its columns for deletion
        if sat_id not in possible_sat_conn_ids:
            cols_to_drop.append(col)

# Drop those columns
df = df.drop(columns=cols_to_drop)

print("Remaining columns after cleanup:", df.columns.tolist())


Remaining columns after cleanup: ['Time', 'GEO1_Name', 'GEO1_Lat', 'GEO1_Lon', 'GEO1_Freq_Hz', 'GEO1_Sydney_Access', 'GEO1_Sydney_SNR_dB', 'GEO1_Sydney_RSSI_dBm', 'GEO1_Melbourne_Access', 'GEO1_Melbourne_SNR_dB', 'GEO1_Melbourne_RSSI_dBm', 'LEO20_Name', 'LEO20_Lat', 'LEO20_Lon', 'LEO20_Freq_Hz', 'LEO20_Sydney_Access', 'LEO20_Sydney_SNR_dB', 'LEO20_Sydney_RSSI_dBm', 'LEO21_Name', 'LEO21_Lat', 'LEO21_Lon', 'LEO21_Freq_Hz', 'LEO21_Sydney_Access', 'LEO21_Sydney_SNR_dB', 'LEO21_Sydney_RSSI_dBm', 'LEO21_Melbourne_Access', 'LEO21_Melbourne_SNR_dB', 'LEO21_Melbourne_RSSI_dBm', 'LEO22_Name', 'LEO22_Lat', 'LEO22_Lon', 'LEO22_Freq_Hz', 'LEO22_Sydney_Access', 'LEO22_Sydney_SNR_dB', 'LEO22_Sydney_RSSI_dBm', 'LEO22_Melbourne_Access', 'LEO22_Melbourne_SNR_dB', 'LEO22_Melbourne_RSSI_dBm', 'LEO23_Name', 'LEO23_Lat', 'LEO23_Lon', 'LEO23_Freq_Hz', 'LEO23_Sydney_Access', 'LEO23_Sydney_SNR_dB', 'LEO23_Sydney_RSSI_dBm', 'LEO23_Melbourne_Access', 'LEO23_Melbourne_SNR_dB', 'LEO23_Melbourne_RSSI_dBm', 'LEO24_N

In [54]:
all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]

In [55]:
for i, row in df.iterrows():
    # Get the highest value across all RSSI columns at the `i-th` row
    highest_rssi_at_i = df[all_rssi_cols].iloc[i].max()

    # Get the column name that has the highest RSSI value at the `i-th` index
    column_with_highest_rssi = df[all_rssi_cols].iloc[i].idxmax()

    print(f"Highest RSSI value at index {i}: {highest_rssi_at_i}")
    print(f"Column with the highest RSSI value at index {i}: {column_with_highest_rssi}")

Highest RSSI value at index 0: -246.902671192829
Column with the highest RSSI value at index 0: LEO717_Sydney_RSSI_dBm
Highest RSSI value at index 1: -140.737906923875
Column with the highest RSSI value at index 1: LEO717_Sydney_RSSI_dBm
Highest RSSI value at index 2: -342.82446730444
Column with the highest RSSI value at index 2: LEO716_Melbourne_RSSI_dBm
Highest RSSI value at index 3: -200.711468876268
Column with the highest RSSI value at index 3: LEO716_Sydney_RSSI_dBm
Highest RSSI value at index 4: -263.774707364643
Column with the highest RSSI value at index 4: LEO716_Sydney_RSSI_dBm
Highest RSSI value at index 5: -305.704524328001
Column with the highest RSSI value at index 5: LEO715_Melbourne_RSSI_dBm


In [ ]:
stats = df[all_rssi_cols].fillna(0).describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\l

In [ ]:
stats = df[all_rssi_cols].describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: Runtim

In [40]:
stats = df[all_snr_cols].describe().T
stats.to_csv(os.path.join('./', 'snr_stats.csv'))

c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\deols\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: Runtim

In [39]:

# Highest value per column
max_per_column = df[all_rssi_cols].max()

# Highest value among all columns
overall_max = df[all_rssi_cols].max().max()

# Displaying results
print("\nHighest Value per Column:\n", max_per_column)
print("\nHighest Value Among All Columns:", overall_max)


Highest Value per Column:
 GEO1_Sydney_RSSI_dBm        -1247.353270
GEO1_Melbourne_RSSI_dBm     -1300.320538
LEO20_Sydney_RSSI_dBm       -2636.448563
LEO21_Sydney_RSSI_dBm       -2416.362038
LEO21_Melbourne_RSSI_dBm    -2682.304715
                                ...     
LEO718_Melbourne_RSSI_dBm   -1640.411237
LEO719_Sydney_RSSI_dBm      -2287.675599
LEO719_Melbourne_RSSI_dBm   -2618.913861
LEO720_Sydney_RSSI_dBm      -2738.740686
LEO720_Melbourne_RSSI_dBm   -2770.981369
Length: 121, dtype: float64

Highest Value Among All Columns: -140.737906923875


In [7]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build SNR column names
    sydney_SNR_col = f'{sat_id}_Sydney_SNR_dB'
    melbourne_SNR_col = f'{sat_id}_Melbourne_SNR_dB'

    # Check which SNR columns exist
    sydney_exists = sydney_SNR_col in df.columns
    melbourne_exists = melbourne_SNR_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_SNR_col],
            labels=[sydney_SNR_col],
            xlabel='Time (m)',
            ylabel='dBm',
            title=f'SNR for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_SNR_.png",
            folder=tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_SNR_col],
            labels=[melbourne_SNR_col],
            xlabel='Time (m) ',
            ylabel='dBm',
            title=f'SNR for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_SNR__melbourne_.png",
            folder=tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [8]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build RSSI column names
    sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
    melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

    # Check which RSSI columns exist
    sydney_exists = sydney_RSSI_col in df.columns
    melbourne_exists = melbourne_RSSI_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_RSSI_col],
            labels=[sydney_RSSI_col],
            xlabel='Time (m)',
            ylabel='dBm',
            title=f'RSSI for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_RSSI_.png",
            folder=tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_RSSI_col],
            labels=[melbourne_RSSI_col],
            xlabel='Time (m) ',
            ylabel='dBm',
            title=f'RSSI for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_RSSI__melbourne_.png",
            folder=tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [9]:
# import matplotlib.pyplot as plt
# import re

# all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
# all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]

# # Loop over each satellite in possible_sat_conn_ids
# for sat_id in possible_sat_conn_ids:
#     # Build RSSI column names
#     sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
#     melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

#     # Check which RSSI columns exist
#     sydney_exists = sydney_RSSI_col in df.columns
#     melbourne_exists = melbourne_RSSI_col in df.columns

#     # Skip if neither exists
#     if not sydney_exists and not melbourne_exists:
#         continue

#     plt.figure(figsize=(10, 5))
#     plt.title(f'RSSI vs Time for {sat_id}')
#     plt.xlabel('Time')
#     plt.ylabel('RSSI (dB)')

#     if sydney_exists:
#         plt.plot(df['Time'], df[sydney_RSSI_col], label='Sydney', color='blue')
#     if melbourne_exists:
#         plt.plot(df['Time'], df[melbourne_RSSI_col], label='Melbourne', color='green')

#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()
